In [46]:
import time
import json
import requests
import pandas as pd
import plotly.express as px

from enum import Enum
from decimal import Decimal
from pydantic.dataclasses import dataclass

In [44]:
def get_bybit_spot():
    baseurl = "https://api.bybit.com"
    payload = { "category": "spot" }
    r = requests.get(baseurl + "/v5/market/tickers", params=payload)
    data = r.json()["result"]["list"]

    markets = []
    for pair in data:
        market = {"symbol" : pair["symbol"], "price": Decimal(pair["lastPrice"]), "volume": Decimal(pair["volume24h"])}
        markets.append(market)

    for market in markets:
        market["rvolume"] = market["price"] * market["volume"]

    markets.sort(key=lambda x: x.get("rvolume"), reverse=True)
    return markets
    

In [13]:
def get_bybit_futures():
    baseurl = "https://api.bybit.com"
    payload = { "category": "linear" }
    r = requests.get(baseurl + "/v5/market/tickers", params=payload)
    data = r.json()["result"]["list"]

    markets = []
    for pair in data:
        market = {"symbol" : pair["symbol"], "price": Decimal(pair["lastPrice"]), "oi": pair["openInterest"], "fr": pair["fundingRate"], "volume": Decimal(pair["volume24h"])}
        markets.append(market)

    for market in markets:
        market["rvolume"] = market["price"] * market["volume"]

    markets.sort(key=lambda x: x.get("rvolume"), reverse=True)
    return markets

In [26]:
class Timeframe(Enum):
    HOURLY = "1h"
    DAILY = "1d"
    WEEKLY = "1w"
    MONTHLY = "1M"

@dataclass
class Candle:
    time: int
    open: Decimal
    high: Decimal
    low: Decimal
    close: Decimal
    volume: Decimal

def exluded_markets(check: str, excluded: list[str]) -> bool:
    if len(excluded) > 0:
        for item in excluded:
            if check.startswith(item) or check.endswith(item):
                return True
    return False

class Binance():
    BASEURL = "https://api.binance.com"

    def __init__(self) -> None:
        pass

    def markets(self, market: str, exclude: list[str] = []) -> list[str]:
        url = Binance.BASEURL + "/api/v3/ticker/price"
        r = requests.get(url)

        if r.status_code != 200:
            raise requests.exceptions.HTTPError(r.json())

        data = filter(lambda x: x.endswith(market) and not exluded_markets(x, exclude), map(lambda x: x.get("symbol", ""), r.json()))
        return list(data)

    def kline(self, symbol: str, interval: Timeframe) -> list[Candle]:
        url = Binance.BASEURL + "/api/v3/klines"
        payload = { "symbol": symbol, "interval": interval.value }
        r = requests.get(url, params=payload)

        if r.status_code != 200:
            raise requests.exceptions.HTTPError(r.json())

        klines = []
        for kline in r.json():
            candle = Candle(*kline[:6])
            klines.append(candle)

        return klines

In [ ]:
stablecoins = ["TUSD", "BUSD", "USDC", "PAX", "USDP", "DAI", "GUSD", "USDD", "USTC", "UST", "USDS"]

binance = Binance()
markets = binance.markets("USDT", stablecoins)

print(f"Downloading {len(markets)} markets...")
market_data = {}
for i in range(len(markets)):
    if i % 20 == 0:
        time.sleep(5)
    market = markets[i]
    market_data[market] = binance.kline(market, Timeframe.DAILY)
print("Finished downloading.")

In [54]:
btcusdt = pd.DataFrame.from_dict(market_data["BTCUSDT"]).set_index("time")

In [55]:
btcusdt

,open,high,low,close,volume
time,,,,,
1639094400000,47535.90000000,50125.00000000,46852.00000000,47140.54000000,44233.57391000
1639180800000,47140.54000000,49485.71000000,46751.00000000,49389.99000000,28889.19358000
1639267200000,49389.99000000,50777.00000000,48638.00000000,50053.90000000,26017.93421000
1639353600000,50053.90000000,50189.97000000,45672.75000000,46702.75000000,50869.52093000
1639440000000,46702.76000000,48700.41000000,46290.00000000,48343.28000000,39955.98445000
...,...,...,...,...,...
1681862400000,30380.01000000,30413.53000000,28520.00000000,28797.10000000,86575.48656000
1681948800000,28797.10000000,29088.30000000,28010.00000000,28243.65000000,76879.09372000
1682035200000,28243.65000000,28374.02000000,27125.00000000,27262.84000000,77684.76790000
